# Estructura del modelo

### Imports

In [1]:
import os
import librosa
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

## Gestión del Dataset

### Funciones

In [2]:
def audio_to_spectrogram(file_path, sr = 20500, n_fft = 2048, hop_length = 512):
    # Cargar el audio
    y, sr = librosa.load(file_path, sr = sr)
    
    # Calcular el espectrograma (STFT)
    S = librosa.stft(y, n_fft = n_fft, hop_length = hop_length)
    
    # Convertir a magnitudes (la magnitud es el espectrograma)
    spectrogram = np.abs(S)
    
    # Escalar en logaritmo (opcional para mejor visualización y aprendizaje)
    log_spectrogram = librosa.amplitude_to_db(spectrogram, ref = np.max)

    return log_spectrogram

def spectrogram_to_audio_sin_fase(magnitud, sr, n_fft = 2048, hop_length = 512, n_iter = 32):
    # Reconstrucción del audio con Griffin-Lim
    audio_reconstruido = librosa.griffinlim(magnitud, n_iter = n_iter, hop_length = hop_length, n_fft = n_fft)
    
    return audio_reconstruido

def visualize_spectrogram(spectrogram, title = "Spectrogram"):
    plt.figure(figsize=(10, 4))

    # Mostrar el espectrograma con un mapa de colores (viridis o inferno suelen ser útiles)
    plt.imshow(spectrogram, aspect = 'auto', origin = 'lower', cmap = 'viridis')
    plt.colorbar(label = "Decibels (dB)")
    plt.title(title)
    plt.xlabel("Time (frames)")
    plt.ylabel("Frequency (bins)")
    plt.tight_layout()
    
    plt.show()

def pad_or_trim(spectrogram, max_length = 94):
    if spectrogram.shape[1] > max_length:  # Recortar
        return spectrogram[:, :max_length]
    else:  # Rellenar
        padding = np.zeros((spectrogram.shape[0], max_length - spectrogram.shape[1]))
        return np.hstack((spectrogram, padding))


### Carga, clasificación y estructuración de los datos

In [ ]:

path = "path"

vocab = []
word_to_index = {}
index_to_word = {}
data = []
flat_data = []
num = 0

max_length = 94
max_height = 1025

for word in os.listdir(path):
    print(num)
    vocab.append(word)

    word_path = os.path.join(path, word)
    spectrogram_list = []
    
    for index, audio in enumerate(os.listdir(word_path)):
        if index > 100:
            break
        
        audio_path = os.path.join(word_path, audio)
        spectrogram = audio_to_spectrogram(audio_path)

        # Ajustar dimensiones
        spectrogram = pad_or_trim(spectrogram)

        spectrogram_list.append((spectrogram))

    data.append(spectrogram_list)
    num += 1


word_to_index = {word: index for index, word in enumerate(vocab)}
index_to_word = {idx: word for word, idx in word_to_index.items()}

for i in range(len(data)):
    for j in range(len(data[i])):
        flat_data.append((i, data[i][j]))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


### Creación del Dataset

In [4]:
batch_size = 32
max_length = 94

labels = [item[0] for item in flat_data]  # Extraer etiquetas
spectrograms = [item[1] for item in flat_data]  # Extraer espectrogramas

# Convertir a tensores
label_tensor = tf.convert_to_tensor(labels, dtype=tf.int32)
spectrogram_tensor = tf.convert_to_tensor(spectrograms, dtype=tf.float32)

# Crear el dataset
dataset = tf.data.Dataset.from_tensor_slices((label_tensor, spectrogram_tensor))

# Barajar y dividir en lotes
dataset = dataset.shuffle(len(labels)).batch(batch_size)

In [5]:
print(vocab)
print(word_to_index)

print(spectrograms)

print(f"Valores mínimos y máximos en labels: {min(labels)}, {max(labels)}")
print(f"Vocab size: {len(vocab)}")

['backward', 'bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'follow', 'forward', 'four', 'happy', 'house', 'learn', 'left', 'marvin', 'nine', 'off', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'visual', 'wow', 'yes', 'zero']
{'backward': 0, 'bed': 1, 'bird': 2, 'cat': 3, 'dog': 4, 'down': 5, 'eight': 6, 'five': 7, 'follow': 8, 'forward': 9, 'four': 10, 'happy': 11, 'house': 12, 'learn': 13, 'left': 14, 'marvin': 15, 'nine': 16, 'off': 17, 'one': 18, 'right': 19, 'seven': 20, 'sheila': 21, 'six': 22, 'stop': 23, 'three': 24, 'tree': 25, 'two': 26, 'visual': 27, 'wow': 28, 'yes': 29, 'zero': 30}
[array([[-80., -80., -80., ...,   0.,   0.,   0.],
       [-80., -80., -80., ...,   0.,   0.,   0.],
       [-80., -80., -80., ...,   0.,   0.,   0.],
       ...,
       [-80., -80., -80., ...,   0.,   0.,   0.],
       [-80., -80., -80., ...,   0.,   0.,   0.],
       [-80., -80., -80., ...,   0.,   0.,   0.]]), array([[-80., -80., -80., ...,   0.,   0.,   0.

# Modelo

### Estructura

In [6]:
class TextEncoder(tf.keras.Model):
    def __init__(self, vocab_size, embed_dim):
        super(TextEncoder, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embed_dim)

    def call(self, x):
        return self.embedding(x)

class SpectrogramDecoder(tf.keras.Model):
    def __init__(self, target_dim):
        super(SpectrogramDecoder, self).__init__()
        self.dense = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dense(target_dim, activation=None)  # No activación para espectrogramas sin normalizar
        ])

    def call(self, x):
        return self.dense(x)

class TextToSpectrogram(tf.keras.Model):
    def __init__(self, vocab_size, embed_dim, target_dim):
        super(TextToSpectrogram, self).__init__()
        self.encoder = TextEncoder(vocab_size, embed_dim)
        self.decoder = SpectrogramDecoder(target_dim)

    def call(self, x):
        x = self.encoder(x)
        return self.decoder(x)
    
def create_model(vocab_size, embed_dim, target_dim):
    # Definir el encoder y el decoder dentro del modelo funcional
    input_text = tf.keras.Input(shape=(None,), name="text_input")  # Entrada de texto
    embedding = tf.keras.layers.Embedding(vocab_size, embed_dim)(input_text)

    # Decoder
    dense_1 = tf.keras.layers.Dense(256, activation='relu')(embedding)
    output_flat = tf.keras.layers.Dense(target_dim, activation=None, name="output")(dense_1)

    # Ajustar dimensiones al formato (1025, 94)
    output_spectrogram = tf.keras.layers.Reshape((max_height, max_length))(output_flat)
    
    # Crear modelo
    model = tf.keras.Model(inputs=input_text, outputs=output_spectrogram, name="TextToSpectrogram")
    return model


# Entrenamiento del Modelo

### Configuración

In [7]:
vocab_size = len(vocab) # Tamaño del vocabulario
embed_dim = 128  # Dimensión del embedding
target_dim = max_length * max_height  # Dimensiones del espectrograma

model = create_model(vocab_size, embed_dim, target_dim)
model.summary()


Model: "TextToSpectrogram"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_input (InputLayer)     [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 128)         3968      
                                                                 
 dense (Dense)               (None, None, 256)         33024     
                                                                 
 output (Dense)              (None, None, 96350)       24761950  
                                                                 
 reshape (Reshape)           (None, 1025, 94)          0         
                                                                 
Total params: 24,798,942
Trainable params: 24,798,942
Non-trainable params: 0
_________________________________________________________________


### Compilación con Funciones de Pérdida

In [8]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="mse",  # Error cuadrático medio
    metrics=["mae"]  # Error absoluto medio como métrica adicional
)

### Entrenamiento

In [9]:
epochs = 10

history = model.fit(dataset, batch_size = batch_size, epochs = epochs)

Epoch 1/10
91/91 [==============================] - 22s 217ms/step - loss: 2361.5291 - mae: 31.4392
Epoch 2/10
91/91 [==============================] - 20s 210ms/step - loss: 2347.2593 - mae: 31.3410
Epoch 3/10
91/91 [==============================] - 20s 207ms/step - loss: 2278.3894 - mae: 30.8585
Epoch 4/10
91/91 [==============================] - 20s 208ms/step - loss: 2093.2822 - mae: 29.5193
Epoch 5/10
91/91 [==============================] - 20s 206ms/step - loss: 1763.6089 - mae: 26.9749
Epoch 6/10
91/91 [==============================] - 20s 206ms/step - loss: 1319.3162 - mae: 23.1241
Epoch 7/10
91/91 [==============================] - 19s 206ms/step - loss: 848.8676 - mae: 18.2879
Epoch 8/10
91/91 [==============================] - 20s 207ms/step - loss: 459.8337 - mae: 13.2577
Epoch 9/10
91/91 [==============================] - 19s 205ms/step - loss: 216.3387 - mae: 9.0306
Epoch 10/10
91/91 [==============================] - 20s 208ms/step - loss: 103.7667 - mae: 6.1098
